# Knowledge Graph Construction with Neo4j and Local Embeddings

This notebook constructs a semantic knowledge graph from user interest data using Neo4j as the graph database and sentence-transformers for local embeddings. The knowledge graph captures relationships between semantic categories, topics, entities, and search clusters.

## 1. Install Required Dependencies

In [2]:
import subprocess
import sys

# Install required packages
packages = [
    'neo4j',
    'sentence-transformers',
    'pandas',
    'numpy',
    'networkx',
    'matplotlib',
    'scikit-learn',
    'tqdm'
]

for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("✓ All dependencies installed successfully")

✓ All dependencies installed successfully


In [3]:
# Import required libraries
import json
import os
from pathlib import Path
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Neo4j imports
from neo4j import GraphDatabase, basic_auth
from neo4j.exceptions import ServiceUnavailable

# Embedding and ML imports
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Visualization imports
import matplotlib.pyplot as plt
import networkx as nx

print("✓ All imports successful")

✓ All imports successful


## 1.5 Docker Commands for Neo4j

Before connecting to Neo4j, ensure the Docker container is running. Use these commands to manage the Neo4j container.

In [ ]:
# Check Neo4j logs
!docker logs neo4j --tail 50

In [ ]:
# Remove Neo4j container (this will delete all data)
!docker stop neo4j
!docker rm neo4j

In [ ]:
# Stop Neo4j container
!docker stop neo4j

In [ ]:
# Start existing Neo4j container (if already created but stopped)
!docker start fabric-kg

print("⏳ Waiting for Neo4j to start...")
import time
time.sleep(10)
print("✓ Neo4j started")

In [ ]:
# Start Neo4j container with increased memory settings
!docker run -d \
  --name fabric-kg \
  -p 7687:7687 \
  -p 7474:7474 \
  -e NEO4J_AUTH=neo4j/password \
  -e NEO4J_dbms_memory_heap_initial__size=2G \
  -e NEO4J_dbms_memory_heap_max__size=2G \
  -e NEO4J_dbms_memory_pagecache_size=1G \
  neo4j:latest

print("⏳ Waiting 20-30 seconds for Neo4j to fully start...")
import time
time.sleep(25)
print("✓ Neo4j should be ready now")

In [ ]:
# Check if Neo4j container is running
!docker ps | grep neo4j

### Access Neo4j Browser

Once the container is running, you can access the Neo4j Browser interface at:

**http://localhost:7474**

Login credentials:
- Username: `neo4j`
- Password: `password`

## 2. Initialize Neo4j Connection

Configure and establish connection to a local Neo4j database instance. Make sure Neo4j is running locally (typically on `bolt://localhost:7687` for default).

In [ ]:
# Neo4j Configuration
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"  # Default password for fresh Neo4j

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = None
        try:
            self.driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
            # Test connection
            with self.driver.session() as session:
                result = session.run("RETURN 1")
                print("✓ Successfully connected to Neo4j")
        except Exception as e:
            error_msg = str(e)
            if "credentials expired" in error_msg.lower():
                print("⚠ Neo4j credentials expired. Attempting to update password...")
                try:
                    # Try with initial password
                    temp_driver = GraphDatabase.driver(uri, auth=basic_auth(user, "neo4j"))
                    with temp_driver.session() as temp_session:
                        temp_session.run(f"ALTER CURRENT USER SET PASSWORD FROM 'neo4j' TO '{password}'")
                    temp_driver.close()
                    # Now try with new password
                    self.driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
                    with self.driver.session() as session:
                        session.run("RETURN 1")
                    print("✓ Successfully updated credentials and connected to Neo4j")
                except Exception as pwd_error:
                    print(f"✗ Failed to update password: {pwd_error}")
                    raise
            elif "ServiceUnavailable" in str(type(e)):
                print("✗ Neo4j service unavailable at", uri)
                print("  Make sure Neo4j is running. You can start it with:")
                print("  - Docker: docker run -it --rm -p 7687:7687 -v data:/data neo4j")
                raise
            else:
                print(f"✗ Connection error: {e}")
                raise
    
    def close(self):
        if self.driver:
            self.driver.close()
    
    def query(self, query_str, **kwargs):
        with self.driver.session() as session:
            return session.run(query_str, **kwargs).data()

# Initialize connection (handle connection errors gracefully)
try:
    neo4j_conn = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
except Exception as e:
    print(f"Connection failed: {e}")
    neo4j_conn = None

## 3. Load Both Data Sources

Load both the user interests data and website extraction cache to build a unified knowledge graph.

In [ ]:
# Load user interest data
json_file_path = Path("structured_user_interests.json")
if not json_file_path.exists():
    json_file_path = Path("/Users/shreyasjagannath/dev/fabric/onfabric-data-science-interview/structured_user_interests_v1.json")

with open(json_file_path, 'r', encoding='utf-8') as f:
    user_data = json.load(f)

print("=== USER INTERESTS DATA ===")
print(f"✓ Data loaded successfully")
print(f"  - Total activities: {user_data['metadata']['total_activities']}")
print(f"  - Total searches: {user_data['metadata']['total_searches']}")
print(f"  - Total page visits: {user_data['metadata']['total_page_visits']}")
print(f"  - Semantic categories: {len(user_data['semantic_categories'])}")
print(f"  - Top entities: {len(user_data['top_entities'])}")

# Load website extraction cache
cache_file = Path("openai_extraction_cache.json")
if not cache_file.exists():
    cache_file = Path("/Users/shreyasjagannath/dev/fabric/onfabric-data-science-interview/openai_extraction_cache_latest.json")

with open(cache_file, 'r', encoding='utf-8') as f:
    website_cache = json.load(f)

print(f"\n=== WEBSITE EXTRACTION DATA ===")
print(f"✓ Data loaded successfully")
print(f"  - Total websites: {len(website_cache)}")

=== USER INTERESTS DATA ===
✓ Data loaded successfully
  - Total activities: 53031
  - Total searches: 30535
  - Total page visits: 22496
  - Semantic categories: 10
  - Top entities: 20

=== WEBSITE EXTRACTION DATA ===
✓ Data loaded successfully
  - Total websites: 19791


## 4. Setup Local Embedding Model

Initialize sentence-transformers for local embeddings. Using `all-MiniLM-L6-v2` (lightweight, ~90MB) for efficient local processing.

In [5]:
# Initialize embedding model
print("Loading embedding model... (this may take a moment on first run)")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"✓ Model loaded: {embedding_model.get_sentence_embedding_dimension()}-dimensional embeddings")

# Embedding function
def get_embedding(text):
    """Generate embedding for a text string"""
    if not text or not isinstance(text, str):
        return None
    try:
        # Clean text
        clean_text = text.strip()[:512]  # Truncate to 512 chars
        embedding = embedding_model.encode(clean_text)
        return embedding.tolist()
    except Exception as e:
        print(f"Error embedding text: {e}")
        return None

# Test embedding
test_embedding = get_embedding("Technology and innovation")
print(f"✓ Test embedding generated: shape {np.array(test_embedding).shape}")

Loading embedding model... (this may take a moment on first run)
✓ Model loaded: 384-dimensional embeddings
✓ Test embedding generated: shape (384,)


## 5. Create Unified Knowledge Graph Schema

Define a unified schema with common node types:
- **Category**: All categories (from both sources)
- **Topic**: All topics (from both sources)  
- **Entity**: All entities/items (from both sources)
- **Website**: Visited websites
- **User**: The user node

In [ ]:
if neo4j_conn:
    # Clear existing data
    print("Clearing existing graph data...")
    try:
        neo4j_conn.query("MATCH (n) DETACH DELETE n")
        print("✓ Existing data cleared")
    except Exception as e:
        print(f"Note: {e}")
    
    # Create constraints for unified schema
    print("Creating constraints...")
    constraints = [
        "CREATE CONSTRAINT category_name IF NOT EXISTS FOR (c:Category) REQUIRE c.name IS UNIQUE",
        "CREATE CONSTRAINT topic_id IF NOT EXISTS FOR (t:Topic) REQUIRE t.topic_id IS UNIQUE",
        "CREATE CONSTRAINT entity_id IF NOT EXISTS FOR (e:Entity) REQUIRE e.entity_id IS UNIQUE",
        "CREATE CONSTRAINT user_id IF NOT EXISTS FOR (u:User) REQUIRE u.user_id IS UNIQUE",
        "CREATE CONSTRAINT website_domain IF NOT EXISTS FOR (w:Website) REQUIRE w.domain IS UNIQUE",
    ]
    
    for constraint in constraints:
        try:
            neo4j_conn.query(constraint)
        except Exception as e:
            pass  # Constraint may already exist
    
    print("✓ Schema constraints created")
    print("\n=== UNIFIED SCHEMA ===")
    print("Node Types:")
    print("  - Category: All categories from both data sources")
    print("  - Topic: All topics from both data sources")
    print("  - Entity: All entities/items from both data sources")
    print("  - Website: Visited websites")
    print("  - User: User profile")
    print("\nRelationship Types:")
    print("  - (Topic)-[:BELONGS_TO]->(Category)")
    print("  - (Topic)-[:MENTIONS]->(Entity)")
    print("  - (User)-[:INTERESTED_IN]->(Topic)")
    print("  - (User)-[:VISITED]->(Website)")
    print("  - (Website)-[:HAS_CATEGORY]->(Category)")
    print("  - (Website)-[:HAS_TOPIC]->(Topic)")
    print("  - (Website)-[:MENTIONS]->(Entity)")
else:
    print("✗ Cannot create schema: Neo4j connection not available")

## 6. Extract and Unify Data from Both Sources

Extract categories, topics, and entities from both data sources into unified dictionaries.

In [ ]:
def clean_text(text):
    """Clean and normalize text"""
    if not text:
        return ""
    if isinstance(text, (list, dict)):
        return str(text)[:256]
    text = ' '.join(str(text).split())
    return text[:256]

def extract_url_domain(text):
    """Extract domain from URL"""
    match = re.search(r'https?://([^/]+)', text)
    return match.group(1) if match else None

def make_id(text):
    """Create a clean ID from text"""
    return text.lower().replace(' ', '_').replace('-', '_')[:100]

print("=== EXTRACTING DATA FROM BOTH SOURCES ===\n")

# Unified dictionaries
all_categories = {}  # category_name -> {name, source, count}
all_topics = {}      # topic_id -> {id, content, category, source, website}
all_entities = {}    # entity_id -> {id, name, type, source, count}
all_websites = {}    # domain -> {domain, url, category, topic, items}

# ============ PROCESS USER INTERESTS DATA ============
print("Processing user interests data...")

# Extract categories from user interests
for category_name, category_data in user_data['semantic_categories'].items():
    cat_id = make_id(category_name)
    all_categories[cat_id] = {
        'id': cat_id,
        'name': category_name,
        'source': 'user_interests',
        'activity_count': category_data.get('activity_count', 0),
        'search_count': category_data.get('search_count', 0)
    }
    
    # Extract topics from this category
    for topic_text in category_data.get('topics', []):
        if not topic_text or not isinstance(topic_text, str) or len(topic_text) < 5:
            continue
        
        topic_text = clean_text(topic_text)
        topic_id = f"user_{make_id(topic_text[:50])}_{len(all_topics)}"
        
        all_topics[topic_id] = {
            'id': topic_id,
            'content': topic_text,
            'category': category_name,
            'source': 'user_interests',
            'is_url': 'http' in topic_text.lower()
        }

# Extract entities from user interests
for entity_name, mention_count in user_data['top_entities'].items():
    entity_id = make_id(entity_name)
    if entity_id not in all_entities:
        all_entities[entity_id] = {
            'id': entity_id,
            'name': entity_name,
            'source': 'user_interests',
            'mention_count': mention_count,
            'entity_type': 'named_entity'
        }

print(f"  ✓ Categories: {len([c for c in all_categories.values() if c['source'] == 'user_interests'])}")
print(f"  ✓ Topics: {len([t for t in all_topics.values() if t['source'] == 'user_interests'])}")
print(f"  ✓ Entities: {len([e for e in all_entities.values() if e['source'] == 'user_interests'])}")

# ============ PROCESS WEBSITE EXTRACTION DATA ============
print("\nProcessing website extraction data...")

for cache_key, extraction_data in website_cache.items():
    # Extract URL from cache key
    url = None
    if 'Website:' in cache_key:
        url = cache_key.split('Website: ')[1].split('\\nContent')[0].strip()
    
    if not url:
        continue
    
    domain = extract_url_domain(url) or url
    category_name = extraction_data.get('category', 'Uncategorized')
    topic_text = extraction_data.get('topic', '')
    items = extraction_data.get('items', [])
    
    # Store website
    all_websites[domain] = {
        'domain': domain,
        'url': url,
        'category': category_name,
        'topic': topic_text,
        'items': items
    }
    
    # Add category if not exists
    cat_id = make_id(category_name)
    if cat_id not in all_categories:
        all_categories[cat_id] = {
            'id': cat_id,
            'name': category_name,
            'source': 'website_extraction',
            'activity_count': 0,
            'search_count': 0
        }
    
    # Add topic
    if topic_text and len(topic_text) > 2:
        topic_id = f"web_{make_id(topic_text[:50])}_{len(all_topics)}"
        all_topics[topic_id] = {
            'id': topic_id,
            'content': topic_text,
            'category': category_name,
            'source': 'website_extraction',
            'website': domain,
            'is_url': False
        }
    
    # Add entities/items
    for item in items:
        if item and isinstance(item, str):
            entity_id = make_id(item)
            if entity_id not in all_entities:
                all_entities[entity_id] = {
                    'id': entity_id,
                    'name': item,
                    'source': 'website_extraction',
                    'mention_count': 1,
                    'entity_type': 'extracted_item'
                }
            else:
                # Increment count if already exists
                all_entities[entity_id]['mention_count'] = all_entities[entity_id].get('mention_count', 0) + 1

print(f"  ✓ Websites: {len(all_websites)}")
print(f"  ✓ New Categories: {len([c for c in all_categories.values() if c['source'] == 'website_extraction'])}")
print(f"  ✓ New Topics: {len([t for t in all_topics.values() if t['source'] == 'website_extraction'])}")
print(f"  ✓ New Entities: {len([e for e in all_entities.values() if e['source'] == 'website_extraction'])}")

print(f"\n=== TOTAL UNIFIED DATA ===")
print(f"  • Total Categories: {len(all_categories)}")
print(f"  • Total Topics: {len(all_topics)}")
print(f"  • Total Entities: {len(all_entities)}")
print(f"  • Total Websites: {len(all_websites)}")

## 7. Generate Embeddings for All Nodes

Generate vector embeddings for all categories, topics, and entities from both data sources.

In [ ]:
print("Generating embeddings for all nodes...")

# Generate embeddings for categories
print("  - Embedding categories...")
for cat_id in tqdm(all_categories.keys()):
    text = all_categories[cat_id]['name']
    all_categories[cat_id]['embedding'] = get_embedding(text)

# Generate embeddings for topics
print("  - Embedding topics...")
for topic_id in tqdm(all_topics.keys()):
    text = all_topics[topic_id]['content']
    all_topics[topic_id]['embedding'] = get_embedding(text)

# Generate embeddings for entities
print("  - Embedding entities...")
for entity_id in tqdm(all_entities.keys()):
    text = all_entities[entity_id]['name']
    all_entities[entity_id]['embedding'] = get_embedding(text)

print("\n✓ Embeddings generated successfully")
print(f"  - Embedding dimension: 384")
print(f"  - Categories with embeddings: {len(all_categories)}")
print(f"  - Topics with embeddings: {len(all_topics)}")
print(f"  - Entities with embeddings: {len(all_entities)}")

## 8. Populate Neo4j with Unified Knowledge Graph

Create all nodes and relationships in Neo4j using the unified schema.

In [ ]:
if neo4j_conn:
    print("Populating Neo4j with unified knowledge graph...")
    
    # Create User node
    print("\n1. Creating User node...")
    user_query = """
    CREATE (u:User {
        user_id: 'user_001',
        total_activities: $total_activities,
        total_searches: $total_searches,
        total_page_visits: $total_page_visits,
        created_at: datetime()
    })
    """
    neo4j_conn.query(user_query, 
        total_activities=user_data['metadata']['total_activities'],
        total_searches=user_data['metadata']['total_searches'],
        total_page_visits=user_data['metadata']['total_page_visits']
    )
    print("  ✓ User node created")
    
    # Create Category nodes
    print("\n2. Creating Category nodes...")
    for cat_id, cat_data in tqdm(all_categories.items(), desc="Categories"):
        cat_query = """
        CREATE (c:Category {
            category_id: $category_id,
            name: $name,
            source: $source,
            activity_count: $activity_count,
            search_count: $search_count,
            embedding: $embedding
        })
        """
        neo4j_conn.query(cat_query,
            category_id=cat_data['id'],
            name=cat_data['name'],
            source=cat_data['source'],
            activity_count=cat_data.get('activity_count', 0),
            search_count=cat_data.get('search_count', 0),
            embedding=cat_data['embedding']
        )
    print(f"  ✓ Created {len(all_categories)} Category nodes")
    
    # Create Entity nodes
    print("\n3. Creating Entity nodes...")
    for entity_id, entity_data in tqdm(all_entities.items(), desc="Entities"):
        entity_query = """
        CREATE (e:Entity {
            entity_id: $entity_id,
            name: $name,
            source: $source,
            entity_type: $entity_type,
            mention_count: $mention_count,
            embedding: $embedding
        })
        """
        neo4j_conn.query(entity_query,
            entity_id=entity_data['id'],
            name=entity_data['name'],
            source=entity_data['source'],
            entity_type=entity_data.get('entity_type', 'unknown'),
            mention_count=entity_data.get('mention_count', 1),
            embedding=entity_data['embedding']
        )
    print(f"  ✓ Created {len(all_entities)} Entity nodes")
    
    # Create Topic nodes with relationships
    print("\n4. Creating Topic nodes and relationships...")
    for topic_id, topic_data in tqdm(all_topics.items(), desc="Topics"):
        # Create Topic node
        topic_query = """
        CREATE (t:Topic {
            topic_id: $topic_id,
            content: $content,
            source: $source,
            is_url: $is_url,
            embedding: $embedding
        })
        """
        neo4j_conn.query(topic_query,
            topic_id=topic_data['id'],
            content=topic_data['content'],
            source=topic_data['source'],
            is_url=topic_data.get('is_url', False),
            embedding=topic_data['embedding']
        )
        
        # Link Topic to Category
        cat_name = topic_data['category']
        cat_id = make_id(cat_name)
        topic_to_category = """
        MATCH (t:Topic {topic_id: $topic_id})
        MATCH (c:Category {category_id: $category_id})
        CREATE (t)-[:BELONGS_TO]->(c)
        """
        try:
            neo4j_conn.query(topic_to_category, topic_id=topic_data['id'], category_id=cat_id)
        except:
            pass
        
        # Link User to Topic (for user interest topics)
        if topic_data['source'] == 'user_interests':
            user_to_topic = """
            MATCH (u:User {user_id: 'user_001'})
            MATCH (t:Topic {topic_id: $topic_id})
            CREATE (u)-[:INTERESTED_IN]->(t)
            """
            try:
                neo4j_conn.query(user_to_topic, topic_id=topic_data['id'])
            except:
                pass
    
    print(f"  ✓ Created {len(all_topics)} Topic nodes with relationships")
    
    # Create Website nodes with relationships
    print("\n5. Creating Website nodes and relationships...")
    for domain, website_data in tqdm(all_websites.items(), desc="Websites"):
        # Create Website node
        website_query = """
        CREATE (w:Website {
            domain: $domain,
            url: $url,
            visited_at: datetime()
        })
        """
        neo4j_conn.query(website_query, domain=domain, url=website_data['url'])
        
        # Link User to Website
        user_to_website = """
        MATCH (u:User {user_id: 'user_001'})
        MATCH (w:Website {domain: $domain})
        CREATE (u)-[:VISITED]->(w)
        """
        neo4j_conn.query(user_to_website, domain=domain)
        
        # Link Website to Category
        cat_name = website_data['category']
        cat_id = make_id(cat_name)
        website_to_category = """
        MATCH (w:Website {domain: $domain})
        MATCH (c:Category {category_id: $category_id})
        MERGE (w)-[:HAS_CATEGORY]->(c)
        """
        try:
            neo4j_conn.query(website_to_category, domain=domain, category_id=cat_id)
        except:
            pass
        
        # Link Website to Topic (if topic exists)
        if website_data['topic']:
            topic_id = f"web_{make_id(website_data['topic'][:50])}_"
            # Find matching topic
            find_topic = """
            MATCH (t:Topic)
            WHERE t.content = $topic_text AND t.source = 'website_extraction'
            RETURN t.topic_id AS topic_id
            LIMIT 1
            """
            result = neo4j_conn.query(find_topic, topic_text=website_data['topic'])
            if result:
                website_to_topic = """
                MATCH (w:Website {domain: $domain})
                MATCH (t:Topic {topic_id: $topic_id})
                MERGE (w)-[:HAS_TOPIC]->(t)
                """
                neo4j_conn.query(website_to_topic, domain=domain, topic_id=result[0]['topic_id'])
        
        # Link Website to Entities
        for item in website_data['items']:
            if item:
                entity_id = make_id(item)
                website_to_entity = """
                MATCH (w:Website {domain: $domain})
                MATCH (e:Entity {entity_id: $entity_id})
                MERGE (w)-[:MENTIONS]->(e)
                """
                try:
                    neo4j_conn.query(website_to_entity, domain=domain, entity_id=entity_id)
                except:
                    pass
    
    print(f"  ✓ Created {len(all_websites)} Website nodes with relationships")
    
    # Create Topic-Entity relationships based on mentions
    print("\n6. Creating Topic-Entity mention relationships...")
    mention_count = 0
    for topic_id, topic_data in tqdm(all_topics.items(), desc="Mentions"):
        topic_content = topic_data['content'].lower()
        for entity_id, entity_data in all_entities.items():
            entity_name = entity_data['name'].lower()
            # Simple string matching
            if len(entity_name) > 2 and entity_name in topic_content:
                mention_query = """
                MATCH (t:Topic {topic_id: $topic_id})
                MATCH (e:Entity {entity_id: $entity_id})
                MERGE (t)-[:MENTIONS]->(e)
                """
                try:
                    neo4j_conn.query(mention_query, topic_id=topic_id, entity_id=entity_id)
                    mention_count += 1
                except:
                    pass
    
    print(f"  ✓ Created {mention_count} MENTIONS relationships")
    
    print("\n✅ Knowledge graph population complete!")
else:
    print("✗ Cannot populate: Neo4j connection not available")

## 9. Query and Visualize the Unified Knowledge Graph (Optional)


Query the unified graph with Categories, Topics, and Entities from both data sources.

In [37]:
if neo4j_conn:
    # Query 1: Get graph statistics
    print("=== KNOWLEDGE GRAPH STATISTICS ===\n")
    
    stats_query = """
    MATCH (n)
    RETURN 
        count(*) as total_nodes,
        size(collect(distinct labels(n))) as node_types
    """
    result = neo4j_conn.query(stats_query)
    if result:
        print(f"Total nodes: {result[0]['total_nodes']}")
    
    # Node counts by type
    node_type_query = """
    MATCH (n)
    UNWIND labels(n) as label
    RETURN label, count(*) as count
    ORDER BY count DESC
    """
    result = neo4j_conn.query(node_type_query)
    print("\nNodes by type:")
    for row in result:
        print(f"  - {row['label']}: {row['count']}")
    
    # Relationship statistics
    rel_query = """
    MATCH ()-[r]->()
    RETURN type(r) as relationship_type, count(*) as count
    ORDER BY count DESC
    """
    result = neo4j_conn.query(rel_query)
    print("\nRelationships by type:")
    for row in result:
        print(f"  - {row['relationship_type']}: {row['count']}")
    
    print("\n✓ Graph statistics retrieved")

[#D733]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv6Address(('::1', 7687, 0, 0))): OSError('No data')


=== KNOWLEDGE GRAPH STATISTICS ===



Exception ignored in: <function Bolt.__del__ at 0x12c322ca0>
Traceback (most recent call last):
  File "/Users/shreyasjagannath/dev/fabric/.venv/lib/python3.12/site-packages/neo4j/_sync/io/_bolt.py", line 200, in __del__
    def __del__(self):

KeyboardInterrupt: 


ServiceUnavailable: Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv6Address(('::1', 7687, 0, 0)))

In [ ]:
if neo4j_conn:
    # Query 2: Get most connected entities
    print("\n=== TOP ENTITIES BY CONNECTIONS ===\n")
    
    top_entities_query = """
    MATCH (e:Entity)-[r]->()
    RETURN e.name as entity, count(r) as connection_count
    ORDER BY connection_count DESC
    LIMIT 10
    """
    result = neo4j_conn.query(top_entities_query)
    for row in result:
        print(f"  {row['entity']}: {row['connection_count']} connections")
    
    print("\n✓ Top entities retrieved")

In [ ]:
if neo4j_conn:
    # Query 3: Get most active categories
    print("\n=== TOP CATEGORIES BY ACTIVITY ===\n")
    
    top_categories_query = """
    MATCH (c:Category)
    RETURN c.name as category, c.activity_count as activity_count, c.search_count as search_count
    ORDER BY c.activity_count DESC
    LIMIT 10
    """
    result = neo4j_conn.query(top_categories_query)
    for row in result:
        print(f"  {row['category']}")
        print(f"    - Activities: {row['activity_count']}, Searches: {row['search_count']}")
    
    print("\n✓ Top categories retrieved")

In [ ]:
if neo4j_conn:
    # Query 4: Semantic search using embeddings
    print("\n=== SEMANTIC SEARCH EXAMPLE ===\n")
    
    # Get an entity embedding for similarity search
    query_text = "technology artificial intelligence machine learning"
    query_embedding = get_embedding(query_text)
    
    print(f"Query: '{query_text}'")
    print("\nFinding similar entities...")
    
    # Get all entities and their embeddings
    entities_query = """
    MATCH (e:Entity)
    WHERE e.embedding IS NOT NULL
    RETURN e.name as name, e.embedding as embedding
    LIMIT 20
    """
    results = neo4j_conn.query(entities_query)
    
    # Calculate similarities
    similarities = []
    for result in results:
        entity_embedding = np.array(result['embedding'])
        query_vec = np.array(query_embedding).reshape(1, -1)
        entity_vec = entity_embedding.reshape(1, -1)
        similarity = cosine_similarity(query_vec, entity_vec)[0][0]
        similarities.append({
            'entity': result['name'],
            'similarity': float(similarity)
        })
    
    # Sort by similarity
    similarities = sorted(similarities, key=lambda x: x['similarity'], reverse=True)
    
    for i, result in enumerate(similarities[:5]):
        print(f"  {i+1}. {result['entity']}: {result['similarity']:.3f}")
    
    print("\n✓ Semantic search completed")

In [ ]:
if neo4j_conn:
    # Query 5: Get subgraph around a category
    print("\n=== CATEGORY SUBGRAPH ===\n")
    
    category_subgraph_query = """
    MATCH (c:Category {name: "Technology & Innovation"})
    MATCH (t:Topic)-[:BELONGS_TO]->(c)
    MATCH (t)-[:MENTIONS]->(e:Entity)
    RETURN 
        c.name as category,
        count(DISTINCT t) as topic_count,
        count(DISTINCT e) as entity_count,
        collect(DISTINCT e.name)[0..5] as sample_entities
    """
    result = neo4j_conn.query(category_subgraph_query)
    if result:
        row = result[0]
        print(f"Category: {row['category']}")
        print(f"Topics: {row['topic_count']}")
        print(f"Entities: {row['entity_count']}")
        print(f"Sample entities: {row['sample_entities']}")
    
    print("\n✓ Category subgraph retrieved")

In [ ]:
# Visualization: Create a sample network graph
if neo4j_conn:
    print("\n=== CREATING NETWORK VISUALIZATION ===\n")
    
    # Build a networkx graph
    G = nx.DiGraph()
    
    # Add sample nodes and edges from the database
    sample_query = """
    MATCH (n1)-[r]->(n2)
    WHERE labels(n1)[0] IN ['Category', 'Topic', 'Entity']
    RETURN 
        labels(n1)[0] as label1,
        coalesce(n1.name, n1.topic_id, n1.entity_id) as node1,
        labels(n2)[0] as label2,
        coalesce(n2.name, n2.topic_id, n2.entity_id) as node2,
        type(r) as relationship_type
    LIMIT 100
    """
    
    results = neo4j_conn.query(sample_query)
    
    # Build graph
    for result in results:
        node1 = result['node1']
        node2 = result['node2']
        label1 = result['label1']
        label2 = result['label2']
        
        G.add_node(node1, node_type=label1)
        G.add_node(node2, node_type=label2)
        G.add_edge(node1, node2, relationship=result['relationship_type'])
    
    print(f"Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Use spring layout for visualization
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    # Color nodes by type
    node_colors = []
    color_map = {'Category': '#FF6B6B', 'Topic': '#4ECDC4', 'Entity': '#45B7D1'}
    for node in G.nodes():
        node_type = G.nodes[node].get('node_type', 'Unknown')
        node_colors.append(color_map.get(node_type, '#95E1D3'))
    
    # Draw network
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=800, ax=ax, alpha=0.9)
    nx.draw_networkx_edges(G, pos, edge_color='gray', arrows=True, 
                          arrowsize=20, arrowstyle='->', ax=ax, 
                          connectionstyle='arc3,rad=0.1', alpha=0.5, width=1.5)
    
    # Draw labels for high-degree nodes only
    high_degree_nodes = {node: node for node, degree in dict(G.degree()).items() if degree > 2}
    nx.draw_networkx_labels(G, pos, labels=high_degree_nodes, font_size=8, ax=ax)
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#FF6B6B', label='Category'),
        Patch(facecolor='#4ECDC4', label='Topic'),
        Patch(facecolor='#45B7D1', label='Entity')
    ]
    ax.legend(handles=legend_elements, loc='upper left')
    
    ax.set_title('Knowledge Graph Network Visualization\n(Sample Subgraph)', 
                fontsize=14, fontweight='bold')
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    
    print("✓ Visualization complete")

In [ ]:
# Summary and utility functions
print("\n" + "="*50)
print("KNOWLEDGE GRAPH CONSTRUCTION COMPLETE")
print("="*50)

# Define utility functions for future use
def search_by_similarity(query_text, node_type='Entity', limit=5):
    """
    Semantic search in the knowledge graph
    """
    if not neo4j_conn:
        print("Neo4j connection not available")
        return []
    
    query_embedding = get_embedding(query_text)
    
    cypher_query = f"""
    MATCH (n:{node_type})
    WHERE n.embedding IS NOT NULL
    RETURN 
        labels(n)[0] as node_type,
        coalesce(n.name, n.topic_id, n.entity_id) as node_id,
        n.embedding as embedding
    LIMIT {limit * 3}
    """
    
    results = neo4j_conn.query(cypher_query)
    
    # Calculate similarities
    similarities = []
    for result in results:
        node_embedding = np.array(result['embedding'])
        query_vec = np.array(query_embedding).reshape(1, -1)
        node_vec = node_embedding.reshape(1, -1)
        similarity = cosine_similarity(query_vec, node_vec)[0][0]
        similarities.append({
            'node_id': result['node_id'],
            'node_type': result['node_type'],
            'similarity': float(similarity)
        })
    
    # Sort and return top results
    similarities = sorted(similarities, key=lambda x: x['similarity'], reverse=True)
    return similarities[:limit]

def get_entity_connections(entity_name):
    """Get all connections for an entity"""
    if not neo4j_conn:
        print("Neo4j connection not available")
        return {}
    
    query = """
    MATCH (e:Entity {name: $name})
    OPTIONAL MATCH (e)<-[:MENTIONS]-(t:Topic)
    OPTIONAL MATCH (e)-[:CO_OCCURS_WITH]->(other:Entity)
    RETURN 
        e.name as entity,
        count(DISTINCT t) as topic_mentions,
        collect(DISTINCT other.name) as related_entities
    """
    
    result = neo4j_conn.query(query, name=entity_name)
    return result[0] if result else {}

# Example usage
print("\n📊 Available Functions:")
print("  - search_by_similarity(query_text, node_type='Entity', limit=5)")
print("  - get_entity_connections(entity_name)")
print("  - neo4j_conn.query(cypher_query)")

print("\n✨ Knowledge graph ready for exploration!")
print(f"\nAccess at: {NEO4J_URI}")
print(f"Login: {NEO4J_USER} / ****")

In [ ]:
search_by_similarity("blackstone")  # Example function call

In [ ]:
# Search for similar topics by semantic similarity
print("\n=== SEMANTIC TOPIC SEARCH ===\n")

# Define a search query
topic_query_text = "machine learning deep learning neural networks"
print(f"Query: '{topic_query_text}'")
print("Finding similar topics...\n")

# Use the search function but for Topics instead of Entities
topic_results = search_by_similarity(topic_query_text, node_type='Topic', limit=10)

print("Top 10 similar topics:")
for i, result in enumerate(topic_results, 1):
    print(f"{i}. {result['node_id']}: {result['similarity']:.4f}")

# You can also search with other queries:
print("\n" + "="*60)
print("Another example - searching for finance-related topics:")
finance_results = search_by_similarity("stock market investment finance banking", node_type='Topic', limit=5)
for i, result in enumerate(finance_results, 1):
    print(f"{i}. {result['node_id']}: {result['similarity']:.4f}")

In [ ]:
# Deep dive: Get detailed topic information with similarity
print("\n=== DETAILED TOPIC SIMILARITY ANALYSIS ===\n")

# Search for topics and get their full details
def search_topics_detailed(query_text, limit=5):
    """Search for topics and return detailed information including content"""
    
    # Get top similar topics
    results = search_by_similarity(query_text, node_type='Topic', limit=limit)
    
    # Fetch detailed information for each topic
    detailed_results = []
    for result in results:
        topic_query = """
        MATCH (t:Topic {topic_id: $topic_id})
        OPTIONAL MATCH (t)-[:BELONGS_TO]->(c:Category)
        OPTIONAL MATCH (t)-[:MENTIONS]->(e:Entity)
        RETURN 
            t.topic_id as topic_id,
            t.content as content,
            c.name as category,
            count(DISTINCT e) as entity_count,
            collect(DISTINCT e.name) as entities
        """
        try:
            topic_details = neo4j_conn.query(topic_query, topic_id=result['node_id'])
            if topic_details:
                detail = topic_details[0]
                detail['similarity'] = result['similarity']
                detailed_results.append(detail)
        except:
            pass
    
    return detailed_results

# Example usage
search_query = "python programming software development"
detailed_topics = search_topics_detailed(search_query, limit=5)

print(f"Query: '{search_query}'\n")
for i, topic in enumerate(detailed_topics, 1):
    print(f"{i}. {topic['topic_id']} (Similarity: {topic['similarity']:.4f})")
    print(f"   Content: {topic['content'][:80]}...")
    print(f"   Category: {topic['category']}")
    print(f"   Entities mentioned: {topic['entity_count']}")
    if topic['entities']:
        print(f"   Entities: {', '.join(topic['entities'][:3])}")
    print()

In [ ]:
# Get top 5 similar topics with complete details
print("\n=== TOP 5 SIMILAR TOPICS WITH FULL DETAILS ===\n")

query_text = "artificial intelligence machine learning"
# query_text = "bollywood"

print(f"Search Query: '{query_text}'\n")
print("="*80)

# Get top 5 similar topics
top_5_topics = search_topics_detailed(query_text, limit=5)

for idx, topic in enumerate(top_5_topics, 1):
    print(f"\n#{idx} - Topic ID: {topic['topic_id']}")
    print(f"    Similarity Score: {topic['similarity']:.4f}")
    print(f"    Category: {topic['category']}")
    print(f"    Content: {topic['content']}")
    if topic['entities'] and len(topic['entities']) > 0:
        print(f"    Related Entities: {', '.join(topic['entities'])}")
    print("-"*80)

print("\n✓ Top 5 similar topics retrieved successfully")

In [ ]:
# View the unified categories from both data sources
print("=== UNIFIED CATEGORIES FROM BOTH DATA SOURCES ===\n")
for i, (cat_id, cat_data) in enumerate(all_categories.items(), 1):
    print(f"{i}. {cat_data['name']}")
    print(f"   - Source: {cat_data['source']}")
    print(f"   - Activities: {cat_data.get('activity_count', 0)}")
    print(f"   - Searches: {cat_data.get('search_count', 0)}")
    # Count topics for this category
    topic_count = sum(1 for t in all_topics.values() if t.get('category') == cat_data['name'])
    print(f"   - Topics: {topic_count}")
    print()